In [1]:
!pip install transformers datasets rouge_score absl-py nltk

# Import necessary libraries
from datasets import load_dataset, load_metric
from transformers import BartTokenizer, BartForConditionalGeneration
import random

# Load the dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Initialize tokenizer and model
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Sample 5 examples for few-shot learning
few_shot_sample_indices = random.sample(range(100), 5)  # Using 100 to ensure enough data
few_shot_examples = [
    {
        "article": dataset['train'][i]['article'],
        "summary": dataset['train'][i]['highlights']
    }
    for i in few_shot_sample_indices
]

# Function to create few-shot prompt
def create_few_shot_prompt(article, examples):
    prompt = "Here are some examples of article summaries:\n"
    for ex in examples:
        prompt += f"Article: {ex['article'][:500]}...\nSummary: {ex['summary']}\n\n"  # Truncate long articles for prompt
    prompt += f"Now, summarize the following article:\n{article[:500]}..."  # Truncate long articles for prompt
    return prompt

# Function to create zero-shot prompt
def create_zero_shot_prompt(article):
    prompt = f"Summarize the following article:\n{article[:500]}..."  # Truncate long articles for prompt
    return prompt

# Sample test articles
test_sample_indices = range(10, 15)
test_articles = [dataset['test'][i]['article'] for i in test_sample_indices]
test_references = [dataset['test'][i]['highlights'] for i in test_sample_indices]

# Generate few-shot prompts for test articles
few_shot_prompts = [create_few_shot_prompt(article, few_shot_examples) for article in test_articles]
# Generate zero-shot prompts for test articles
zero_shot_prompts = [create_zero_shot_prompt(article) for article in test_articles]

# Generate summaries using few-shot prompts
few_shot_summaries = []
for prompt in few_shot_prompts:
    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    few_shot_summaries.append(summary)

# Generate summaries using zero-shot prompts
zero_shot_summaries = []
for prompt in zero_shot_prompts:
    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    zero_shot_summaries.append(summary)

# Initialize ROUGE metric
rouge = load_metric("rouge")

# Evaluate the generated summaries using ROUGE
few_shot_results = []
for summary, reference in zip(few_shot_summaries, test_references):
    result = rouge.compute(predictions=[summary], references=[reference])
    few_shot_results.append(result)

zero_shot_results = []
for summary, reference in zip(zero_shot_summaries, test_references):
    result = rouge.compute(predictions=[summary], references=[reference])
    zero_shot_results.append(result)

# Display results
print("Few-shot learning results:")
for i, result in enumerate(few_shot_results):
    print(f"\nTest Article {i+1}:")
    print(f"Generated Summary: {few_shot_summaries[i]}")
    print(f"Reference Summary: {test_references[i]}")
    print(f"ROUGE Scores: {result}")

print("Zero-shot learning results:")
for i, result in enumerate(zero_shot_results):
    print(f"\nTest Article {i+1}:")
    print(f"Generated Summary: {zero_shot_summaries[i]}")
    print(f"Reference Summary: {test_references[i]}")
    print(f"ROUGE Scores: {result}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 98.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.2/417.2 kB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 123.5 MB/s eta 0:00:

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

/tmp/ipykernel_7917/771616405.py:66: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y
Few-shot learning results:

Test Article 1:
Generated Summary: "Transplant tourists" travel to poor countries to buy organs from the desperate. Pakistan, where trade in human organs is legal, is turning into a "kidney bazaar" O.J. Simpson will be held without bail after his arrest on robbery and assault charges.
Reference Summary: London's Metropolitan Police say the man was arrested at Luton airport after landing on a flight from Istanbul .
He's been charged with terror offenses allegedly committed since the start of November .
ROUGE Scores: {'rouge1': AggregateScore(low=Score(precision=0.11904761904761904, recall=0.15151515151515152, fmeasure=0.13333333333333333), mid=Score(precision=0.